<a href="https://colab.research.google.com/github/beyzassevval/deneme/blob/main/brain_mr_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
# Adım 1: Gerekli Kütüphaneleri Yükleme ve Kaggle API Kurulumu
# ==============================================================================
print("Adım 1: Kurulumlar yapılıyor...")
# Kaggle API ile kolayca veri seti indirmek için opendatasets kütüphanesini yüklüyoruz.
!pip install opendatasets --upgrade --quiet
# TensorFlow ve Keras'ı yüklüyoruz.
!pip install tensorflow --quiet

import opendatasets as od
import os
import shutil
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from google.colab import files
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd

print("\nLütfen Kaggle API anahtarınızı (kaggle.json) yükleyin.")
# Kullanıcıdan kaggle.json dosyasını yüklemesini isteyen arayüz
# Bu kod çalıştırıldığında bir "Dosya Seç" butonu çıkacaktır.
uploaded = files.upload()

# Yüklenen dosyanın adını kontrol et
if 'kaggle.json' in uploaded:
    print("\n'kaggle.json' başarıyla yüklendi.")
    # Dosyayı doğru konuma taşıyalım
    if not os.path.exists('/root/.kaggle'):
        os.makedirs('/root/.kaggle')
    shutil.move('kaggle.json', '/root/.kaggle/kaggle.json')
    # İzinleri ayarla
    os.chmod('/root/.kaggle/kaggle.json', 600)
else:
    raise ValueError("HATA: Lütfen 'kaggle.json' dosyasını yüklediğinizden emin olun.")

Adım 1: Kurulumlar yapılıyor...

Lütfen Kaggle API anahtarınızı (kaggle.json) yükleyin.


Saving kaggle.json to kaggle.json

'kaggle.json' başarıyla yüklendi.


In [ ]:
# ==============================================================================
# Adım 2: Kaggle Veri Setini İndirme
# ==============================================================================
print("Adım 2: Veri seti indiriliyor...")

dataset_url = "https://www.kaggle.com/datasets/navoneel/brain-mri-images-for-brain-tumor-detection"
od.download(dataset_url, data_dir="brain_mri_dataset")

print("Veri seti indirildi.")


Adım 2: Veri seti indiriliyor...
Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: beyzasenkus
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/navoneel/brain-mri-images-for-brain-tumor-detection


100%|██████████| 15.1M/15.1M [00:00<00:00, 762MB/s]

Veri seti indirildi.


In [ ]:
import os

# Veri seti klasörünü buraya yaz (senin indirdiğin yol)
data_path = "/content/brain_mri_dataset/brain-mri-images-for-brain-tumor-detection"

# Klasör yapısını göster
for root, dirs, files in os.walk(data_path):
    level = root.replace(data_path, '').count(os.sep)
    indent = ' ' * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 4 * (level + 1)
    for f in files[:5]:  # İlk 5 dosyayı göster
        print(f"{subindent}{f}")
    if len(files) > 5:
        print(f"{subindent}... ({len(files)} dosya)")

# Sınıf başına kaç görsel var
print("\nSınıf başına dosya sayısı:")
for class_name in os.listdir(os.path.join(data_path, '/content/brain_mri_dataset/brain-mri-images-for-brain-tumor-detection/brain_tumor_dataset')):
    class_dir = os.path.join(data_path, '/content/brain_mri_dataset/brain-mri-images-for-brain-tumor-detection/brain_tumor_dataset', class_name)
    if os.path.isdir(class_dir):
        count = len(os.listdir(class_dir))
        print(f"{class_name}: {count} adet")


brain-mri-images-for-brain-tumor-detection/
    no/
        23 no.jpg
        46 no.jpg
        no 100.jpg
        21 no.jpg
        no 3.jpg
        ... (98 dosya)
    brain_tumor_dataset/
        no/
            23 no.jpg
            46 no.jpg
            no 100.jpg
            21 no.jpg
            no 3.jpg
            ... (98 dosya)
        yes/
            Y195.JPG
            Y69.jpg
            Y112.JPG
            Y11.jpg
            Y53.jpg
            ... (155 dosya)
    yes/
        Y195.JPG
        Y69.jpg
        Y112.JPG
        Y11.jpg
        Y53.jpg
        ... (155 dosya)

Sınıf başına dosya sayısı:
no: 98 adet
yes: 155 adet


In [ ]:
# 2️⃣ Veri yolları ve klasörler
base_dir = "/content/brain_mri_dataset/brain-mri-images-for-brain-tumor-detection/brain_tumor_dataset"
train_dir = "data/train"
val_dir = "data/val"
test_dir = "data/test"

for d in [train_dir, val_dir, test_dir]:
    os.makedirs(os.path.join(d, "yes"), exist_ok=True)
    os.makedirs(os.path.join(d, "no"), exist_ok=True)

yes_images = os.listdir(os.path.join(base_dir, "yes"))
no_images = os.listdir(os.path.join(base_dir, "no"))

yes_train, yes_temp = train_test_split(yes_images, test_size=0.3, random_state=42)
yes_val, yes_test = train_test_split(yes_temp, test_size=0.5, random_state=42)
no_train, no_temp = train_test_split(no_images, test_size=0.3, random_state=42)
no_val, no_test = train_test_split(no_temp, test_size=0.5, random_state=42)

def copy_images(img_list, src_folder, dest_folder):
    for img in img_list:
        shutil.copy(os.path.join(src_folder, img), os.path.join(dest_folder, img))

copy_images(yes_train, os.path.join(base_dir, "yes"), os.path.join(train_dir, "yes"))
copy_images(yes_val, os.path.join(base_dir, "yes"), os.path.join(val_dir, "yes"))
copy_images(yes_test, os.path.join(base_dir, "yes"), os.path.join(test_dir, "yes"))

copy_images(no_train, os.path.join(base_dir, "no"), os.path.join(train_dir, "no"))
copy_images(no_val, os.path.join(base_dir, "no"), os.path.join(val_dir, "no"))
copy_images(no_test, os.path.join(base_dir, "no"), os.path.join(test_dir, "no"))


In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8,1.2]
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=1,
    class_mode='binary',
    shuffle=False
)

Found 352 images belonging to 2 classes.
Found 38 images belonging to 2 classes.
Found 39 images belonging to 2 classes.


In [ ]:
# 4️⃣ Model - EfficientNetB0 + fine-tuning
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.models import Model

base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = True  # Fine-tuning için açık

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# 5️⃣ Compile
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# 6️⃣ Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
